In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,990.02,0.00,0.02,0.00,0.00,-0.00,0.00
3,0.00,301.27,-0.00,-0.00,-0.00,0.00,0.00
4,0.02,-0.00,75.46,-0.00,-0.00,-0.00,-0.00
5,0.00,-0.00,-0.00,14.74,-0.00,0.00,-0.00
6,0.00,-0.00,-0.00,-0.00,2.68,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.47,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00006,0.00002,0.00005,-0.00004,0.00006
3,0.00000,1.00000,-0.00002,-0.00005,-0.00004,0.00001,0.00001
4,0.00006,-0.00002,1.00000,-0.00003,-0.00003,-0.00004,-0.00001
5,0.00002,-0.00005,-0.00003,1.00000,-0.00002,0.00001,-0.00002
6,0.00005,-0.00004,-0.00003,-0.00002,1.00000,0.00002,0.00001
7,-0.00004,0.00001,-0.00004,0.00001,0.00002,1.00000,0.00005
8,0.00006,0.00001,-0.00001,-0.00002,0.00001,0.00005,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.02643868391073885

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.40437435e+05 1.26064904e+04 7.59050993e+02 2.71969127e+01
 8.00592107e-01 1.85044801e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998374,-0.055562,-0.012483,-0.002424,-0.000438,-0.000078,-0.000008
1,0.054785,0.996987,-0.054008,-0.009794,-0.001776,-0.000311,-0.000034
2,0.015270,0.052658,0.997140,-0.051227,-0.008881,-0.001559,-0.000173
3,0.003693,0.012177,0.049925,0.996637,-0.062825,-0.010555,-0.001173
4,0.000886,0.002926,0.011679,0.061063,0.993922,-0.090289,-0.009584
5,0.000235,0.000769,0.003073,0.015722,0.087824,0.989711,-0.111810
6,0.000052,0.000172,0.000689,0.003526,0.019393,0.110479,0.993683


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00162585, 0.00301328, 0.00285988, 0.00336278, 0.00607764,
       0.0102888 , 0.00631732])